**Setup**

In [9]:
!pip install transformers
!pip install sentencepiece
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Import packages**

In [10]:
import random
import pandas as pd
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from datasets import load_dataset
from IPython.display import display, HTML

**Load Dataset and split into {train,test,validation}**

In [11]:
model_size="base"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-snli-entailment"
dataset = load_dataset("persiannlp/parsinlu_entailment")
parsinlu_train = load_dataset('persiannlp/parsinlu_entailment', split='train')
parsinlu_test = load_dataset('persiannlp/parsinlu_entailment', split='test')
parsinlu_valid = load_dataset('persiannlp/parsinlu_entailment', split='validation')
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

Reusing dataset parsinlu_reading_comprehension (/root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)


  0%|          | 0/3 [00:00<?, ?it/s]

Reusing dataset parsinlu_reading_comprehension (/root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)
Reusing dataset parsinlu_reading_comprehension (/root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)
Reusing dataset parsinlu_reading_comprehension (/root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)
You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


**Display random examples (train data)**

In [12]:
def display_random_examples(dataset=parsinlu_train, num_examples=4):
    assert num_examples < len(dataset)
    
    random_picks = []
    for i in range(num_examples):
        random_pick = random.randint(0,len(dataset)-1)
        random_picks.append(random_pick)
    
    df = pd.DataFrame(dataset[random_picks])
    display(HTML(df.to_html()))
        
display_random_examples(parsinlu_train, 10)

,sent1,sent2,category,label
0,Unbar ، فرانسیسکو! او به زبان اسپانیایی صدا زد.,او فرانسیسکو را به زبان اسپانیایی صدا می کند.,translation-train,e
1,و همانطور که احتمالاً می دانید ، در ارائه تخمین های نیازهای آینده خود ، سرویس پستی از داده های مالی و عملیاتی سال مالی ۱۹۹۸ به عنوان معیار استفاده کرد.,سرویس پستی قادر به یافتن داده های مالی و عملیاتی نبود.,translation-train,c
2,من فکر نمی کنم آنها حس مسئوليت داشته باشند.,من فکر نمی کنم آنها احساس مسئوليت کنند که دیگران را ایمن نگه دارند.,translation-train,n
3,همچنین ، اگر مالیاتهای غیر مستقیم ایالتی و فدرال در سطح فعلی باقی بمانند ، به دلیل کاهش فروش ، درآمد مالیاتی با گذشت زمان کاهش می یابد.,مالیاتهای غیر مستقیم ایالتی و فدرال کمتر از یک سال در این سطح بوده اند.,translation-train,n
4,مرگ او به علت سرطان ریه بود.اگر چه در تاریخ اجراهای تلویزیونی به نام کافن به عنوان یک کمدین نگریسته می‌شود، اما او خود را یک کمدین نمی‌دانست و خود را با نام «مرد رقص و آواز» لقب داده بود.,او همواره خود را از برترین کمدین‌های تاریخ می‌دانست ولی در جوانی در اثر سرطان در گذشت.,natural-wiki,c
5,حدود سال‌های ۱۳۵۰ سید محمود طالقانی اجازۀ دریافت وجوهات برای کمک به مردم نیازمند و انجام فعالیت‌های مددکاری را به منصوریان داد.,طالقانی در دهۀ پنجاه، یکی از روحانیون محبوب آن زمان بود.,natural-wiki,n
6,واتیکان دوم باعث ایجاد یک مذهب کاتولیک کمتر سلسله مراتبی و بیرونی تر شد و زمینه ای برای نوآوری های غیرقابل تصور مانند اجازه پوشیدن لباس شخصی به راهبه ها و برگزاری مراسم به زبان انگلیسی و سایر زبان های مدرن فراهم آورد.,واتیکان دوم باعث تمرکز بیشتر قدرت و ساختار سلسله مراتبی شد.,translation-dev,c
7,درحمله ای جداگانه در بزرگراه پايتخت به فرودگاه مردان مسلح به دو اتومبيل که احتمالا حامل پيمانکاران غير نظامی بود، حمله کردند.,صبح امروز در فرودگاه پايتخت پرواز های زیادی انجام شد.,natural-voa,n
8,این خنده دار است,این یک داستان مضحک است.,translation-dev,e
9,نقاشانی که به تعامل نور با مدل‌ها و محیط اطرافشان اهمیت بسزایی می‌دادند.,نقش نور در نقاشی شان بسیار مهم بود.,natural-wiki,e


**XLM-RoBERTa**

In [5]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='xlm-roberta-base')
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
# forward pass
output = model(**encoded_input)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

**ParsBERT**

In [6]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 
# Define the model repo
model_name = "persiannlp/parsbert-base-parsinlu-entailment" 
# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Transform input tokens 
inputs = tokenizer("Hello world!", return_tensors="pt")
# Model apply
outputs = model(**inputs)

Downloading:   0%|          | 0.00/928 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

Some weights of the model checkpoint at persiannlp/parsbert-base-parsinlu-entailment were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/457 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

**Building a model**

In [13]:
def run_model(premise, hypothesis, **generator_args):
    input_ids = tokenizer.encode(f"{premise}<sep>{hypothesis}", return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    #print(output)
    return output

**Evaluating the model**

In [14]:
Predicted_Class = ''
Real_Class = ''
sent1	= ''
sent2	= ''
TP = 0
FN = 0
for Test in range(0,parsinlu_test.num_rows):
  sent1 = parsinlu_test['sent1'][Test]
  sent2 = parsinlu_test['sent2'][Test]
  Real_Class = parsinlu_test['label'][Test]
  Predicted_Class = run_model(sent1,sent2,)
  if (Predicted_Class[0] == Real_Class):
    TP = TP + 1
  else:
    FN = FN + 1
print('Accuracy using XLM-RoBERTa is : ',((TP) / (TP+FN))*100,' %')

Accuracy using XLM-RoBERTa is :  60.17910447761194  %
